# チャットモデルとプロンプトテンプレートを使ったシンプルなLLMアプリケーションの構築

このクイックスタートでは、LangChainを使ってシンプルなLLMアプリケーションを構築する方法を説明します。このアプリケーションは、英語のテキストを他の言語に翻訳します。これは比較的シンプルなLLMアプリケーションで、LLM呼び出し1回とプロンプトをいくつか実行するだけです。それでも、LangChainを使い始めるには最適な方法です。プロンプトとLLM呼び出しだけで、多くの機能を構築できます。

このチュートリアルを読めば、以下の概要を理解できます。

- [言語モデル](/docs/concepts/chat_models)の使用

- [プロンプトテンプレート](/docs/concepts/prompt_templates)の使用

- [LangSmith](https://docs.smith.langchain.com/)を使ったアプリケーションのデバッグとトレース

さあ、始めましょう！

## セットアップ

### Jupyter Notebook

このチュートリアルやその他のチュートリアルは、[Jupyter Notebook](https://jupyter.org/) で実行するのが最も便利です。インタラクティブな環境でガイドを進めることで、より深く理解することができます。インストール方法については、[こちら](https://jupyter.org/install) をご覧ください。

### インストール

LangChain をインストールするには、次のコマンドを実行します:

<!-- HIDE_IN_NB
import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>
HIDE_IN_NB -->

In [ ]:
# | output: false

# %pip install langchain
# OR
# %conda install langchain -c conda-forge

詳細については、[インストール ガイド](/docs/how_to/installation)をご覧ください。

### LangSmith

LangChain で構築するアプリケーションの多くは、複数のステップで LLM 呼び出しを複数回実行します。
これらのアプリケーションが複雑になるにつれて、チェーンまたはエージェント内で何が起こっているかを正確に検査することが重要になります。
これを行うには、[LangSmith](https://smith.langchain.com) を使用するのが最適です。

上記のリンクでサインアップしたら、環境変数を設定してトレースのログ記録を開始してください。

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
export LANGSMITH_PROJECT="default" # or any other project name
```

または、ノートブックの場合は次のように設定できます:

```python
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
```

## 言語モデルの使用

まずは、言語モデルを単体で使う方法を学びましょう。LangChainは、相互に利用可能な様々な言語モデルをサポートしています。特定のモデルの使用開始方法の詳細については、[サポートされている統合](/docs/integrations/chat/)をご覧ください。

<!-- HIDE_IN_NB>
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4o-mini"}}} />
HIDE_IN_NB -->

In [ ]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

まずはモデルを直接使ってみましょう。[ChatModels](/docs/concepts/chat_models)はLangChainの[Runnables](/docs/concepts/runnables/)のインスタンスであり、これらを操作するための標準インターフェースを公開しています。モデルを呼び出すには、[messages](/docs/concepts/messages/)のリストを`.invoke`メソッドに渡します。

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-32654a56-627c-40e1-a141-ad9350bbfd3e-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

:::tip

LangSmith を有効にしている場合、この実行が LangSmith に記録され、[LangSmith トレース](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r) を確認できます。LangSmith トレースは、[トークン](/docs/concepts/tokens/) の使用情報、レイテンシ、[標準モデルパラメータ](/docs/concepts/chat_models/#standard-parameters) (温度など)、その他の情報を報告します。

:::

ChatModelsは[message](/docs/concepts/messages/)オブジェクトを入力として受け取り、メッセージオブジェクトを出力として生成することに注意してください。メッセージオブジェクトは、テキストコンテンツに加えて、会話における[role](/docs/concepts/messages/#role)を伝え、[tool call](/docs/concepts/tool_calling/)やトークンの使用回数などの重要なデータを保持します。

LangChainは、文字列または[OpenAI format](/docs/concepts/messages/#openai-format)によるチャットモデルの入力もサポートしています。以下は同等です。

```python
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])
```

### ストリーミング

チャットモデルは[Runnables](/docs/concepts/runnables/)であるため、非同期およびストリーミング呼び出しモードを含む標準インターフェースを公開しています。これにより、チャットモデルから個々のトークンをストリーミングできます。

In [ ]:
for token in model.stream(messages):
    print(token.content, end="|")

|C|iao|!||

チャット モデル出力のストリーミングの詳細については、[このガイド](/docs/how_to/chat_streaming/) を参照してください。

## プロンプトテンプレート

現在、メッセージのリストを言語モデルに直接渡しています。このメッセージのリストはどこから来るのでしょうか？通常、これはユーザー入力とアプリケーションロジックの組み合わせから構築されます。このアプリケーションロジックは通常、生のユーザー入力を受け取り、言語モデルに渡す準備が整ったメッセージのリストに変換します。一般的な変換には、システムメッセージの追加や、ユーザー入力に基づいたテンプレートのフォーマット設定などがあります。

[プロンプトテンプレート](/docs/concepts/prompt_templates/)は、この変換を支援するために設計されたLangChainの概念です。生のユーザー入力を受け取り、言語モデルに渡す準備が整ったデータ（プロンプト）を返します。

ここでプロンプトテンプレートを作成しましょう。2つのユーザー変数を受け取ります。

- `language`: テキストを翻訳する言語
- `text`: 翻訳するテキスト

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

`ChatPromptTemplate` は、1 つのテンプレートで複数の [メッセージロール](/docs/concepts/messages/#role) をサポートしていることに注意してください。`language` パラメータはシステムメッセージに、ユーザーの `text` はユーザーメッセージにフォーマットされます。

このプロンプトテンプレートへの入力は辞書です。このプロンプトテンプレートを単体で操作して、どのような動作をするか確認してみましょう。

In [ ]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

2つのメッセージからなる`ChatPromptValue`が返されていることがわかります。メッセージに直接アクセスしたい場合は、次のようにします。

In [ ]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

最後に、フォーマットされたプロンプトでチャット モデルを呼び出すことができます。

In [ ]:
response = model.invoke(prompt)
print(response.content)

Ciao!


:::tip

メッセージの「コンテンツ」には、テキストと[コンテンツブロック](/docs/concepts/messages/#aimessage)、そして追加の構造を含めることができます。詳細については、[こちらのガイド](/docs/how_to/output_parser_string/)をご覧ください。

:::

[LangSmith トレース](https://smith.langchain.com/public/3ccc2d5e-2869-467b-95d6-33a577df99a2/r) を見ると、チャット モデルが受け取るプロンプトのほか、[トークン](/docs/concepts/tokens/) の使用情報、レイテンシ、[標準モデル パラメータ](/docs/concepts/chat_models/#standard-parameters) (温度など)、その他の情報も正確に確認できます。

## まとめ

これで終わりです！このチュートリアルでは、初めてのシンプルなLLMアプリケーションの作成方法を学びました。言語モデルの扱い方、プロンプトテンプレートの作成方法、そしてLangSmithで作成したアプリケーションに優れた可観測性を取り入れる方法も学びました。

これは、優れたAIエンジニアになるために学ぶべきことのほんの一部に過ぎません。幸いなことに、他にもたくさんのリソースをご用意しています！

LangChainのコアコンセプトについてさらに詳しく知りたい方は、詳細な[概念ガイド](/docs/concepts)をご覧ください。

これらの概念についてより具体的なご質問がある場合は、ハウツーガイドの以下のセクションをご覧ください。

- [チャットモデル](/docs/how_to/#chat-models)
- [プロンプトテンプレート](/docs/how_to/#prompt-templates)

LangSmith のドキュメントもご覧ください。

- [LangSmith](https://docs.smith.langchain.com)